In [1]:
import sqlite3 as sql3
import pandas as pd
import csv

In [ ]:
csv_escolas = 'MICRODADOS_ENEM_ESCOLA.csv'
csv_idhm_municipios = 'idhm-municipios.csv'
db_file_path = 'tp2.db'

# --- PADRONIZANDO OS NOMES DAS TABELAS --- Jean, melhor não redeclarar essas variáveis depois pra gente não confundir
tabela_escolas = 'escola'
tabela_idhm_municipios = 'idhm_municipio' 
tabela_ibge_dtb = 'municipios_ibge_dtb'
# Mapeamento de nome completo da UF para sigla 
uf_nome_para_sigla = {
    'Rondônia': 'RO', 'Acre': 'AC', 'Amazonas': 'AM', 'Roraima': 'RR', 'Pará': 'PA',
    'Amapá': 'AP', 'Tocantins': 'TO', 'Maranhão': 'MA', 'Piauí': 'PI', 'Ceará': 'CE',
    'Rio Grande do Norte': 'RN', 'Paraíba': 'PB', 'Pernambuco': 'PE', 'Alagoas': 'AL',
    'Sergipe': 'SE', 'Bahia': 'BA', 'Minas Gerais': 'MG', 'Espírito Santo': 'ES',
    'Rio de Janeiro': 'RJ', 'São Paulo': 'SP', 'Paraná': 'PR', 'Santa Catarina': 'SC',
    'Rio Grande do Sul': 'RS', 'Mato Grosso do Sul': 'MS', 'Mato Grosso': 'MT',
    'Goiás': 'GO', 'Distrito Federal': 'DF'
}


try:
    # --- LEITURA E PREPARAÇÃO DOS DATAFRAMES ---

    df_escolas = pd.read_csv(csv_escolas, encoding='utf-8', sep=';')

    df_idhm_municipios = pd.read_csv(csv_idhm_municipios, encoding='utf-8', sep=',')

    file_path_ibge_dtb = 'RELATORIO_DTB_BRASIL_2024_MUNICIPIOS.csv' 
    df_ibge_dtb = pd.read_csv(
        file_path_ibge_dtb,
        encoding='utf-8', 
        sep=',',
        skiprows=6
    )
    df_ibge_dtb['UF_Sigla'] = df_ibge_dtb['Nome_UF'].map(uf_nome_para_sigla)
    # Renomear colunas para o SQLite para evitar problemas com espaços ou caracteres especiais
    df_ibge_dtb = df_ibge_dtb.rename(columns={
        'Região Geográfica Intermediária': 'Regiao_Geografica_Intermediaria',
        'Nome Região Geográfica Intermediária': 'Nome_Regiao_Geografica_Intermediaria',
        'Região Geográfica Imediata': 'Regiao_Geografica_Imediata',
        'Nome Região Geográfica Imediata': 'Nome_Regiao_Geografica_Imediata',
        'Código Município Completo': 'Codigo_Municipio_Completo', 
        'Nome_Município': 'Nome_Municipio' 
    })
    if 'Unnamed: 9' in df_ibge_dtb.columns:
        df_ibge_dtb = df_ibge_dtb.drop(columns=['Unnamed: 9'])


    # --- CRIAÇÃO/RECRIAÇÃO DE TABELAS E INSERÇÃO DE DADOS ---
    conn = sql3.connect(db_file_path)
    cursor = conn.cursor()

    create_table_escolas_query = f"""
    CREATE TABLE {tabela_escolas} (
        NU_ANO INTEGER NOT NULL,
        CO_UF_ESCOLA INTEGER,
        SG_UF_ESCOLA CHAR(2),
        CO_MUNICIPIO_ESCOLA INTEGER,
        NO_MUNICIPIO_ESCOLA TEXT,
        CO_ESCOLA_EDUCACENSO INTEGER,
        NO_ESCOLA_EDUCACENSO TEXT,
        TP_DEPENDENCIA_ADM_ESCOLA CHAR(1),
        TP_LOCALIZACAO_ESCOLA CHAR(1),
        NU_MATRICULAS INTEGER,
        NU_PARTICIPANTES_NEC_ESP INTEGER,
        NU_PARTICIPANTES INTEGER,
        NU_TAXA_PARTICIPACAO FLOAT,
        NU_MEDIA_CN FLOAT,
        NU_MEDIA_CH FLOAT,
        NU_MEDIA_LP FLOAT,
        NU_MEDIA_MT FLOAT,
        NU_MEDIA_RED FLOAT,
        NU_MEDIA_OBJ FLOAT,
        NU_MEDIA_TOT FLOAT,
        INSE TEXT,
        PC_FORMACAO_DOCENTE FLOAT,
        NU_TAXA_PERMANENCIA FLOAT,
        NU_TAXA_APROVACAO FLOAT,
        NU_TAXA_REPROVACAO FLOAT,
        NU_TAXA_ABANDONO FLOAT,
        PORTE_ESCOLA TEXT,
        CONSTRAINT check_TP_DEPENDENCIA_ADM_ESCOLA CHECK (TP_DEPENDENCIA_ADM_ESCOLA IN ('1','2','3','4','5','6','7','8','9')),\n
        CONSTRAINT check_TP_LOCALIZACAO_ESCOLA CHECK (TP_LOCALIZACAO_ESCOLA IN ('1','2')),\n
        CONSTRAINT pk_escola PRIMARY KEY(NU_ANO, CO_ESCOLA_EDUCACENSO)\n
    );
    """
    create_table_idhm_municipios_query = f"""
    CREATE TABLE idhm_municipio (
    "Territorialidades" VARCHAR(255),
    "IDHM 2000" DECIMAL(4, 3),
    "IDHM 2010" DECIMAL(4, 3),
    "IDHM Renda 2000" DECIMAL(4, 3),
    "IDHM Renda 2010" DECIMAL(4, 3),
    "IDHM Longevidade 2000" DECIMAL(4, 3),
    "IDHM Longevidade 2010" DECIMAL(4, 3),
    "IDHM Educação 2000" DECIMAL(4, 3),
    "IDHM Educação 2010" DECIMAL(4, 3),
    "Subíndice de frequência escolar - IDHM Educação 2000" DECIMAL(4, 3),
    "Subíndice de frequência escolar - IDHM Educação 2010" DECIMAL(4, 3),
    "Subíndice de escolaridade - IDHM Educação 2000" DECIMAL(4, 3),
    "Subíndice de escolaridade - IDHM Educação 2010" DECIMAL(4, 3),
    "Renda per capita 2012" DECIMAL(10, 2),
    "Renda per capita 2013" DECIMAL(10, 2),
    "Renda per capita 2014" DECIMAL(10, 2),
    "Renda per capita 2015" DECIMAL(10, 2),
    "Subíndice de escolaridade - IDHM Educação 2012" DECIMAL(4, 3),
    "Subíndice de escolaridade - IDHM Educação 2013" DECIMAL(4, 3),
    "Subíndice de escolaridade - IDHM Educação 2014" DECIMAL(4, 3),
    "Subíndice de escolaridade - IDHM Educação 2015" DECIMAL(4, 3),
    "Subíndice de frequência escolar - IDHM Educação 2012" DECIMAL(4, 3),
    "Subíndice de frequência escolar - IDHM Educação 2013" DECIMAL(4, 3),
    "Subíndice de frequência escolar - IDHM Educação 2014" DECIMAL(4, 3),
    "Subíndice de frequência escolar - IDHM Educação 2015" DECIMAL(4, 3),
    "IDHM Educação 2012" DECIMAL(4, 3),
    "IDHM Educação 2013" DECIMAL(4, 3),
    "IDHM Educação 2014" DECIMAL(4, 3),
    "IDHM Educação 2015" DECIMAL(4, 3),
    "IDHM Longevidade 2012" DECIMAL(4, 3),
    "IDHM Longevidade 2013" DECIMAL(4, 3),
    "IDHM Longevidade 2014" DECIMAL(4, 3),
    "IDHM Longevidade 2015" DECIMAL(4, 3),
    "IDHM Renda 2012" DECIMAL(4, 3),
    "IDHM Renda 2013" DECIMAL(4, 3),
    "IDHM Renda 2014" DECIMAL(4, 3),
    "IDHM Renda 2015" DECIMAL(4, 3),
    "IDHM 2012" DECIMAL(4, 3),
    "IDHM 2013" DECIMAL(4, 3),
    "IDHM 2014" DECIMAL(4, 3),
    "IDHM 2015" DECIMAL(4, 3),
    "Renda per capita 2000" DECIMAL(10, 2),
    "Renda per capita 2010" DECIMAL(10, 2),
    "Desagregação HOMEM IDHM Censo" DECIMAL(4, 3), -- Assumindo que "Censo_1" e "Censo_2" foram simplificados para um nome comum
    "Desagregação HOMEM IDHM Censo.1" DECIMAL(4, 3), -- Para manter o .1, .2, etc. se forem colunas separadas
    "Desagregação HOMEM IDHM Renda Censo" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Renda Censo.1" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Longevidade Censo" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Longevidade Censo.1" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Educação Censo" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Educação Censo.1" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de frequência escolar - IDHM Educação Censo" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de frequência escolar - IDHM Educação Censo.1" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de escolaridade - IDHM Educação Censo" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de escolaridade - IDHM Educação Censo.1" DECIMAL(4, 3),
    "Desagregação HOMEM Renda per capita PNAD" DECIMAL(10, 2),
    "Desagregação HOMEM Renda per capita PNAD.1" DECIMAL(10, 2),
    "Desagregação HOMEM Renda per capita PNAD.2" DECIMAL(10, 2),
    "Desagregação HOMEM Renda per capita PNAD.3" DECIMAL(10, 2),
    "Desagregação HOMEM Subíndice de escolaridade - IDHM Educação PNAD" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de escolaridade - IDHM Educação PNAD.1" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de escolaridade - IDHM Educação PNAD.2" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de escolaridade - IDHM Educação PNAD.3" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de frequência escolar - IDHM Educação PNAD" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de frequência escolar - IDHM Educação PNAD.1" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de frequência escolar - IDHM Educação PNAD.2" DECIMAL(4, 3),
    "Desagregação HOMEM Subíndice de frequência escolar - IDHM Educação PNAD.3" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Educação PNAD" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Educação PNAD.1" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Educação PNAD.2" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Educação PNAD.3" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Longevidade PNAD" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Longevidade PNAD.1" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Longevidade PNAD.2" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Longevidade PNAD.3" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Renda PNAD" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Renda PNAD.1" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Renda PNAD.2" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM Renda PNAD.3" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM PNAD" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM PNAD.1" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM PNAD.2" DECIMAL(4, 3),
    "Desagregação HOMEM IDHM PNAD.3" DECIMAL(4, 3),
    "Desagregação HOMEM Renda per capita Censo" DECIMAL(10, 2),
    "Desagregação HOMEM Renda per capita Censo.1" DECIMAL(10, 2),
    "Desagregação MULHER IDHM Censo" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Censo.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Renda Censo" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Renda Censo.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Longevidade Censo" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Longevidade Censo.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Educação Censo" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Educação Censo.1" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de frequência escolar - IDHM Educação Censo" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de frequência escolar - IDHM Educação Censo.1" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de escolaridade - IDHM Educação Censo" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de escolaridade - IDHM Educação Censo.1" DECIMAL(4, 3),
    "Desagregação MULHER Renda per capita PNAD" DECIMAL(10, 2),
    "Desagregação MULHER Renda per capita PNAD.1" DECIMAL(10, 2),
    "Desagregação MULHER Renda per capita PNAD.2" DECIMAL(10, 2),
    "Desagregação MULHER Renda per capita PNAD.3" DECIMAL(10, 2),
    "Desagregação MULHER Subíndice de escolaridade - IDHM Educação PNAD" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de escolaridade - IDHM Educação PNAD.1" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de escolaridade - IDHM Educação PNAD.2" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de escolaridade - IDHM Educação PNAD.3" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de frequência escolar - IDHM Educação PNAD" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de frequência escolar - IDHM Educação PNAD.1" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de frequência escolar - IDHM Educação PNAD.2" DECIMAL(4, 3),
    "Desagregação MULHER Subíndice de frequência escolar - IDHM Educação PNAD.3" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Educação PNAD" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Educação PNAD.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Educação PNAD.2" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Educação PNAD.3" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Longevidade PNAD" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Longevidade PNAD.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Longevidade PNAD.2" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Longevidade PNAD.3" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Renda PNAD" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Renda PNAD.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Renda PNAD.2" DECIMAL(4, 3),
    "Desagregação MULHER IDHM Renda PNAD.3" DECIMAL(4, 3),
    "Desagregação MULHER IDHM PNAD" DECIMAL(4, 3),
    "Desagregação MULHER IDHM PNAD.1" DECIMAL(4, 3),
    "Desagregação MULHER IDHM PNAD.2" DECIMAL(4, 3),
    "Desagregação MULHER IDHM PNAD.3" DECIMAL(4, 3),
    "Desagregação MULHER Renda per capita Censo" DECIMAL(10, 2),
    "Desagregação MULHER Renda per capita Censo.1" DECIMAL(10, 2),
    CONSTRAINT pk_idhm_municipio PRIMARY KEY ("Territorialidades")
    );
    """
    create_table_ibge_dtb_query = f"""
    CREATE TABLE IF NOT EXISTS {tabela_ibge_dtb} (
        "Grande Região" TEXT,
        "Código UF" INTEGER,
        "Unidade da Federação" TEXT,
        "UF_Sigla" TEXT, -- Nova coluna para a sigla da UF
        "Código Mesorregião" INTEGER,
        "Nome da Mesorregião" TEXT,
        "Código Microrregião" INTEGER,
        "Nome da Microrregião" TEXT,
        "Código Município Completo" INTEGER PRIMARY KEY,
        "Nome do Município" TEXT
    );
    """
    
    # Criando conexão com sql
    conn = sql3.connect(db_file_path)
    cursor = conn.cursor()
    cursor.execute(f"DROP TABLE IF EXISTS {tabela_escolas}")
    cursor.execute(f"DROP TABLE IF EXISTS {tabela_idhm_municipios}")
    cursor.execute(f"DROP TABLE IF EXISTS {tabela_ibge_dtb}") # Garante que a tabela antiga é dropada
    cursor.execute(create_table_ibge_dtb_query)
    cursor.execute(create_table_escolas_query)
    cursor.execute(create_table_idhm_municipios_query)
    #print(df_escolas.isnull().sum())
    # Colocando o dataframe como tabela no banco de daos
    df_escolas.to_sql(tabela_escolas, conn, if_exists='append', index=False)
    df_idhm_municipios.to_sql(tabela_idhm_municipios, conn, if_exists='append', index=False)
    df_ibge_dtb.to_sql(tabela_ibge_dtb, conn, if_exists='replace', index=False)
    

finally:
    if 'conn' in locals() and conn:
        conn.close()


/tmp/ipykernel_41103/723473343.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_escolas = pd.read_csv(csv_escolas, encoding='utf-8', sep=';')


In [3]:
conn = sql3.connect("tp2.db")
cursor = conn.cursor()
query = "PRAGMA table_info(idhm_municipio);"
df = pd.read_sql(query, conn)
display(df)

,cid,name,type,notnull,dflt_value,pk
0,0,Territorialidades,VARCHAR(255),0,None,1
1,1,IDHM 2000,"DECIMAL(4, 3)",0,None,0
2,2,IDHM 2010,"DECIMAL(4, 3)",0,None,0
3,3,IDHM Renda 2000,"DECIMAL(4, 3)",0,None,0
4,4,IDHM Renda 2010,"DECIMAL(4, 3)",0,None,0
...,...,...,...,...,...,...
122,122,Desagregação MULHER IDHM PNAD.1,"DECIMAL(4, 3)",0,None,0
123,123,Desagregação MULHER IDHM PNAD.2,"DECIMAL(4, 3)",0,None,0
124,124,Desagregação MULHER IDHM PNAD.3,"DECIMAL(4, 3)",0,None,0
125,125,Desagregação MULHER Renda per capita Censo,"DECIMAL(10, 2)",0,None,0


In [4]:
conn = sql3.connect(db_file_path)
try:
    print("--- Contagem de Registros ---")
    query_count_escolas = f"SELECT COUNT(*) FROM {tabela_escolas};"
    df_count_escolas = pd.read_sql(query_count_escolas, conn)
    print(f"Total de registros na tabela '{tabela_escolas}':\n{df_count_escolas}\n")

    query_count_idhm = f"SELECT COUNT(*) FROM {tabela_idhm_municipios};"
    df_count_idhm = pd.read_sql(query_count_idhm, conn)
    print(f"Total de registros na tabela '{tabela_idhm_municipios}':\n{df_count_idhm}\n")

    query_count_dtb = f"SELECT COUNT(*) FROM {tabela_ibge_dtb};"
    df_count_dtb = pd.read_sql(query_count_dtb, conn)
    print(f"Total de registros na tabela '{tabela_ibge_dtb}':\n{df_count_dtb}\n")

except Exception as e:
    print(f"Erro ao contar registros: {e}")
finally:
    if conn:
        conn.close()

--- Contagem de Registros ---
Total de registros na tabela 'escola':
   COUNT(*)
0    172304

Total de registros na tabela 'idhm_municipio':
   COUNT(*)
0      5569

Total de registros na tabela 'municipios_ibge_dtb':
   COUNT(*)
0      5571



In [ ]:
# Conecta ao banco de dados
conn = sql3.connect(db_file_path)
cursor = conn.cursor()

try:
    cursor.execute(f"PRAGMA table_info({tabela_idhm_municipios});")
    columns = [col[1] for col in cursor.fetchall()]
    if 'cod_municipio_ibge' not in columns:
        cursor.execute(f"ALTER TABLE {tabela_idhm_municipios} ADD COLUMN cod_municipio_ibge INTEGER;")
        print("Coluna 'cod_municipio_ibge' adicionada à tabela 'idhm_municipio'.")
    else:
        print("Coluna 'cod_municipio_ibge' já existe.")


    update_query_final = f"""
    UPDATE {tabela_idhm_municipios}
    SET cod_municipio_ibge = (
        SELECT T.Codigo_Municipio_Completo 
        FROM {tabela_ibge_dtb} AS T
        WHERE
            LOWER(TRIM(REPLACE(REPLACE(SUBSTR({tabela_idhm_municipios}.Territorialidades, 1, INSTR({tabela_idhm_municipios}.Territorialidades, ' (') - 1), '''', ''), '-', ''))) = LOWER(TRIM(REPLACE(REPLACE(T.Nome_Municipio, '''', ''), '-', ''))) -- Use T.Nome_Municipio conforme renomeado
            AND
            LOWER(SUBSTR({tabela_idhm_municipios}.Territorialidades, INSTR({tabela_idhm_municipios}.Territorialidades, '(') + 1, 2)) = LOWER(T.UF_Sigla)
    )
    WHERE {tabela_idhm_municipios}.Territorialidades LIKE '%(%)%';
    """
    cursor.execute(update_query_final)
    conn.commit()
    print("Coluna 'cod_municipio_ibge' na tabela 'idhm_municipio' atualizada com sucesso usando o mapeamento IBGE DTB e a nova coluna 'UF_Sigla'.")


except Exception as e:
    print(f"Ocorreu um erro durante a atualização da tabela IDHM: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

Coluna 'cod_municipio_ibge' adicionada à tabela 'idhm_municipio'.
Coluna 'cod_municipio_ibge' na tabela 'idhm_municipio' atualizada com sucesso usando o mapeamento IBGE DTB e a nova coluna 'UF_Sigla'.


In [6]:
conn = sql3.connect(db_file_path)
try:
    print("--- Contagem de Registros ---")
    query_count_escolas = f"SELECT COUNT(*) FROM {tabela_escolas};"
    df_count_escolas = pd.read_sql(query_count_escolas, conn)
    print(f"Total de registros na tabela '{tabela_escolas}':\n{df_count_escolas}\n")

    query_count_idhm = f"SELECT COUNT(*) FROM {tabela_idhm_municipios};"
    df_count_idhm = pd.read_sql(query_count_idhm, conn)
    print(f"Total de registros na tabela '{tabela_idhm_municipios}':\n{df_count_idhm}\n")

    query_count_dtb = f"SELECT COUNT(*) FROM {tabela_ibge_dtb};"
    df_count_dtb = pd.read_sql(query_count_dtb, conn)
    print(f"Total de registros na tabela '{tabela_ibge_dtb}':\n{df_count_dtb}\n")

except Exception as e:
    print(f"Erro ao contar registros: {e}")
finally:
    if conn:
        conn.close()

--- Contagem de Registros ---
Total de registros na tabela 'escola':
   COUNT(*)
0    172304

Total de registros na tabela 'idhm_municipio':
   COUNT(*)
0      5569

Total de registros na tabela 'municipios_ibge_dtb':
   COUNT(*)
0      5571

